In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

sns.set(style="whitegrid")

fig_DIR = "../figs/team_leaders/"
box_DIR = "../data/boxscores_team/"

In [ ]:
from plotnine import ggplot, aes, ggsave, themes, theme
from plotnine import (
    geom_point,
    geom_line,
    geom_smooth,
    geom_hline,
    facet_wrap,
    geom_boxplot,
    geom_violin,
    geom_density,
)
from plotnine import labs, element_rect, element_blank, element_text
from plotnine import (
    scale_color_manual,
    scale_y_continuous,
    ylim,
    scale_x_date,
    scale_color_identity,
)

theme_sra = themes.theme_538(base_size=9, base_family="Tahoma")
theme_sra += theme(
    # plot_background = element_rect(fill = 'ghostwhite', color = "ghostwhite"),
    plot_title=element_text(face="bold", size=16),
    strip_text=element_text(face="bold", size=10),
    plot_caption=element_text(size=10),
    plot_subtitle=element_text(size=12),
    axis_text_x=element_text(size=8),
    axis_text_y=element_text(size=8),
    axis_title_x=element_text(size=12),
    axis_title_y=element_text(size=12),
)

# Team Shooting Splits

In [ ]:
from nba_api.stats.endpoints import leaguedashteamshotlocations

In [ ]:
stats = leaguedashteamshotlocations.LeagueDashTeamShotLocations(
    distance_range="8ft Range", per_mode_detailed="PerGame"
)
df = stats.get_data_frames()[0]
df1 = pd.DataFrame()
df1["tID"] = df[""]["TEAM_ID"]
df1["Team"] = df[""]["TEAM_NAME"]
df1["r00_8"] = df["Less Than 8 ft."]["FGA"]
df1["r08_16"] = df["8-16 ft."]["FGA"]
df1["r16_24"] = df["16-24 ft."]["FGA"]
df1["r24_"] = df["24+ ft."]["FGA"]
df1["sum"] = df1.iloc[:, -4:].sum(axis=1)
df1.iloc[:, -5:-1] = df1.iloc[:, -5:-1].div(df1["sum"], axis=0) * 100
df1.iloc[:, -5:-1] = df1.iloc[:, -5:-1].round(2)
df1 = df1.drop(["sum"], axis=1)
data = df1
data.to_csv("../R/fdata" + "nba_team_shooting_splits.csv")

# Team Trends

In [ ]:
from nba_api.stats.endpoints import leaguegamelog

In [ ]:
season = 2023
df1 = pd.read_csv(box_DIR + f"NBA_BoxScores_Adv_{season}.csv")
# cols = ["gameId","teamId","offensiveRating","defensiveRating","netRating","possessions"]
# df1 = df1[cols]
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=season,
    season_type_all_star="Regular Season",
)
df2 = stats.get_data_frames()[0]
df2 = df2.rename(
    columns={
        "GAME_ID": "gameId",
        "TEAM_ID": "teamId",
        "TEAM_ABBREVIATION": "teamTricode",
        "GAME_DATE": "gameDate",
        "TEAM_NAME": "teamName",
    }
)
cols2 = ["gameId", "teamId", "teamTricode", "gameDate"]
df2 = df2[cols2]
df2["gameId"] = df2["gameId"].astype(int)
df3 = pd.merge(df2, df1, on=["gameId", "teamId", "teamTricode"])
df3["gameDate"] = pd.to_datetime(df3["gameDate"], format="%Y-%m-%d")
df4 = df3.sort_values(by=["teamTricode", "gameDate"]).reset_index(drop=True)
df6 = df3.sort_values(by=["gameDate"]).reset_index(drop=True)
data = df6.copy()

In [ ]:
# data.columns

In [ ]:
team = "WAS"
df = data[data["teamTricode"] == team].reset_index()
var, y = "reboundPercentage", "Rebound %"
# var,y = "Minutes", "Minutes Played"
title = team + " " + y
p = (
    ggplot(df)  # What data to use
    + aes(x="gameDate", y=var)  # What variable to use
    # + geom_line()  # Geometric object to use for drawing
    + geom_point(color="#CE1141")
    + geom_smooth(method="loess", color="black", se=False)
    + scale_x_date(date_labels="%b-%d", date_breaks="1 week")
    # + geom_hline(yintercept=0.752, linetype="dashed", color="blue", size=0.5)
    # + annotate("text", x=df["gameDate"].iloc[1], y=0.76, label="League Avg")
    + labs(x="Date", y=y, title=title, caption="@SravanNBA | source: nba.com/stats")
    # + themes.theme_xkcd(base_size=14)
    + themes.theme_538(base_size=12)
    + theme(plot_title=element_text(face="bold", size=18), plot_margin=0.025)
)
p.save(fig_DIR + team + "_" + var + ".png", dpi=300)
p.draw()